In [1]:
from datasets import DatasetDict, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    T5Config,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
    pipeline,
)
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
import numpy as np
import time
import gc
import GPUtil
import evaluate
from numba import cuda
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import wandb
import os
import pickle
import optuna
from typing import Dict, Union, Optional, Tuple, List, Any
import pandas as pd
import string

2023-11-21 14:45:00.086738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 14:45:00.086801: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 14:45:00.086836: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
wandb.login()
os.environ["WAND_NOTEBOOK_NAME"] = "w266_final_project_models"
os.environ["WANDB_DIR"] = "../models/wandb"
os.environ["WANDB_PROJECT"] = "w266_final_project"

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garykong. Use `wandb login --relogin` to force relogin


In [3]:
# Random seed for reproducibility
RANDOM_SEED = 42

# Parameters for classification
BATCH_SIZE_EVAL = 32
BATCH_SIZE_TRAIN = 32

# Default parameters for T5 model fine-tuning
PER_DEVICE_TRAIN_BATCH_SIZE = 64
PER_DEVICE_EVAL_BATCH_SIZE = 128
LEARNING_RATE = 3e-4
NUM_TRAIN_EPOCHS = 20
EARLY_STOPPING_PATIENCE = 2
NUM_BEAMS = 4

# Setting the DEVICE to cuda
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set path for profane word list
PROFANE_WORD_PATH = "../data/raw/en.txt"

# Set path for raw dataset dictionary
RAW_DATASET_PATH = "../data/processed/raw_dataset.pkl"
AUG_DATASET_ALL_FILTERS_PATH = "../data/processed/aug_datasets_all_filters"
AUG_DATASET_NO_TOXICITY_FILTER_PATH = "../data/processed/aug_datasets_no_toxicity_filter"
AUG_DATASET_NO_SIMILARITY_FILTER_PATH = "../data/processed/aug_datasets_no_similarity_filter"
AUG_DATASET_NO_ACCEPTABILITY_FILTER_PATH = "../data/processed/aug_datasets_no_acceptability_filter"

# Set path for txt file containing best model checkpoints
BEST_MODEL_CHECKPOINT_PATH = "../models/best_model_checkpoints.txt"

# Set maximum length for input and output
MAX_INPUT_LENGTH = 64
MAX_OUTPUT_LENGTH = 64

In [4]:
# Load tokenizers and models
tokenizer_t5_base = T5Tokenizer.from_pretrained("t5-base")
model_t5_base = T5ForConditionalGeneration.from_pretrained("t5-base").to(DEVICE)
tokenizer_t5_small = T5Tokenizer.from_pretrained("t5-small")
model_t5_small = T5ForConditionalGeneration.from_pretrained("t5-small").to(DEVICE)
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier").to(DEVICE)
tokenizer_acceptability = AutoTokenizer.from_pretrained("iproskurina/tda-bert-en-cola")
model_acceptability = AutoModelForSequenceClassification.from_pretrained("iproskurina/tda-bert-en-cola").to(DEVICE)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [5]:
# Load datasets
raw_datasets = DatasetDict.load_from_disk(RAW_DATASET_PATH)
aug_datasets_all_filters = DatasetDict.load_from_disk(AUG_DATASET_ALL_FILTERS_PATH)
aug_datasets_no_acceptability_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_ACCEPTABILITY_FILTER_PATH)
aug_datasets_no_similarity_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_SIMILARITY_FILTER_PATH)
aug_datasets_no_toxicity_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_TOXICITY_FILTER_PATH)

## Debugging Functions

In [6]:
def measure_time(func, *args, **kwargs):
    """
    Calculates the time it takes to run a function.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Function {func.__name__} took {elapsed_time:.2f} seconds to run.")
    return result

def get_gpu_memory():
    """
    Gets the GPU memory information.
    """
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    print(f"Total GPU memory: {gpu.memoryTotal}MB")
    print(f"Free GPU memory: {gpu.memoryFree}MB")
    print(f"Used GPU memory: {gpu.memoryUsed}MB")

def force_clear_GPU_memory():
    """
    Force clears the GPU memory.
    """
    cuda.select_device(0)
    cuda.close()

def cleanup():
    """
    Cleans up the GPU memory.
    """
    gc.collect()
    torch.cuda.empty_cache()

## Evaluation Metrics

In [7]:
# Initialize model variables
model_bleurt = None
model_bertscore = None
model_sacrebleu = None

def calc_sacrebleu(refs, preds):
    """
    Calculates the SacreBLEU score.

    Args:
        refs (list): List of reference sentences
        preds (list): List of predicted sentences
    
    Returns:
        results (float): SacreBLEU score
    """
    global model_sacrebleu

    if model_sacrebleu is None:
        model_sacrebleu = evaluate.load("sacrebleu")

    results = model_sacrebleu.compute(predictions=preds, references=refs)["score"]
    results = results/100

    return results

def calc_bert_score(
    refs, preds, model_type="microsoft/deberta-large-mnli", output_mean=True
    ):
    """
    Calculates BERT score per line. Note: https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0 lists the best performing models
    Args:
        refs (list): List of reference sentences.
        y_pred (list): List of predicted sentences.
        model_type (str): Type of BERT model to use.
        output_mean (bool): Whether to output the mean of the scores.

    Returns:
        list of precision, recall, f1 scores.

    """
    global model_bertscore

    if model_bertscore is None:
        model_bertscore = evaluate.load("bertscore")
        
    results = model_bertscore.compute(predictions=preds, references=refs, model_type=model_type)
    precision = np.array(results["precision"])
    recall = np.array(results["recall"])
    f1 = np.array(results["f1"])
    
    if output_mean:
        precision = precision.mean()
        recall = recall.mean()
        f1 = f1.mean()

    return precision, recall, f1

def calc_bleurt(refs, preds, checkpoint="BLEURT-20_D12", output_mean = True):
    """
    Calculates BLEURT score per line.

    Args:
        refs (list): List of reference sentences.
        preds (list): List of predicted sentences.
        output_type (str): Type of output to return. Either 'numpy' or 'list'.

    Returns:
        list/array of BLEURT scores.
    """
    global model_bleurt

    if model_bleurt is None:
        model_bleurt = evaluate.load("bleurt", module_type="metric", checkpoint=checkpoint)

    results = np.array(model_bleurt.compute(predictions=preds, references=refs)["scores"])

    if output_mean:
        results = results.mean()

    return results

def calc_tox_acceptability(
    data,
    tokenizer,
    model,
    output_score=True,
    output_mean=True):
    """
    Calculates toxicity and acceptability scores for a given dataset.

    Args:
        data = list of strings to be evaluated
        tokenizer = tokenizer for the model
        model = model to be used for evaluation
        output_score = whether to output the score or the label
        output_mean = whether to output the mean of the scores or the scores for each sentence
    
    Returns:
        array of toxicity and acceptability scores.
    """  
    inputs = tokenizer(data, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs)["logits"]
        if output_score:
            result = torch.nn.functional.softmax(logits, dim=1)[:, 1]
        else:
            result = logits.argmax(1).data
        result = result.cpu().numpy()

    if output_mean:
        result = result.mean()
        
    return result

def evaluate_metrics(
    refs,
    preds,
    tokenizer_toxicity=tokenizer_toxicity,
    model_toxicity=model_toxicity,
    tokenizer_acceptability=tokenizer_acceptability,
    model_acceptability=model_acceptability,
    to_neutral=True,
    weights={
        "BLEU": 0.2,
        "STA": 0.4,
        "Acceptability": 0.2,
        "BERT_Score": 0.2
    },
    include_bleurt=False
):
    """
    Calculates and returns a dictionary of evaluation metrics

    Args:
        refs (list): list of strings (reference)
        preds (list): list of strings (predictions)
        tokenizer_toxicity (tokenizer): tokenizer for toxicity model
        model_toxicity (model): toxicity model
        tokenizer_acceptability (tokenizer): tokenizer for acceptability model
        model_acceptability (model): acceptability model
        to_neutral (bool): whether the goal is to transfer to neutral (True) or to toxic (False)
        weights (dict): dictionary of weights for each metric
        include_bleurt (bool): whether to include BLEURT score in the output

    Returns:
        results (dict): dictionary of evaluation metrics
    """
    # Calculate BLEU score
    bleu = calc_sacrebleu(refs, preds)

    # Calculate toxicity classification
    tox_pred = calc_tox_acceptability(preds, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)

    # Calculate style transfer accuracy as proportion of sentences that were correctly classified (as non-toxic / toxic)
    if to_neutral:
        sta_correct_label = 0
    else:
        sta_correct_label = 1

    sta_pred = (tox_pred == sta_correct_label).sum() / len(tox_pred)

    # Calculate acceptability scores
    acc_pred = calc_tox_acceptability(preds, tokenizer_acceptability, model_acceptability)

    # Calculate similarity score
    bert_score_f1 = calc_bert_score(refs, preds, model_type="distilbert-base-uncased")[2]

    # Calculate BLEURT score if include_bleurt is True
    bleurt = None
    if include_bleurt:
        bleurt = calc_bleurt(refs, preds)

    # Calculate composite score
    composite_score = weights["BLEU"] * bleu + weights["STA"] * sta_pred + weights["Acceptability"] * acc_pred + weights["BERT_Score"] * bert_score_f1

    # Return a dictionary of metrics
    results = {
        "BLEU": bleu,
        "STA": sta_pred,
        "FLU": acc_pred,
        "SEM": bert_score_f1,
        "Overall": composite_score,
    }
    if include_bleurt:
        results["BLEURT"] = bleurt
        
    return results

# Baseline Models

In [8]:
def baseline_detoxifier(text_list, profane_word_path=PROFANE_WORD_PATH):
    """
    Returns a detoxified version of the text by replacing toxic terms with blanks

    Args:
        text_list (list): list of strings to be detoxified
        toxic_list (list): list of toxic terms to be removed from text_list

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    # Load list of profane words
    profane_words = []
    with open(profane_word_path, "r") as f:
        for line in f:
            profane_words.append(line.strip())

    # Detoxify text
    y_pred_delete = []
    for text in text_list:
        for term in profane_words:
            text = text.replace(term, "")
        y_pred_delete.append(text)

    return y_pred_delete

def bart_detoxifier(text_list):
    """
    Returns a detoxified version of the text using BART

    Args:
        text_list (list): list of strings to be detoxified

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    pipe_bart = pipeline("text2text-generation", model="s-nlp/bart-base-detox", device=DEVICE)
    y_pred_bart = pipe_bart(text_list, max_length=MAX_OUTPUT_LENGTH, truncation=True)
    y_pred_bart = [x["generated_text"] for x in y_pred_bart]
    
    return y_pred_bart

In [9]:
# Evaluate DELETE model on validation set
delete_preds_val = baseline_detoxifier(raw_datasets["validation"]['source'])
delete_val_metrics = evaluate_metrics(raw_datasets["validation"]['target'], delete_preds_val)
delete_val_metrics

{'BLEU': 0.5291006187073797,
 'STA': 0.6596814752724225,
 'FLU': 0.47865131,
 'SEM': 0.9118211839944499,
 'Overall': 0.6477872136441012}

In [10]:
# Evaluate BART model on validation set
bart_preds_val = bart_detoxifier(raw_datasets["validation"]['source'])
bart_val_metrics = evaluate_metrics(raw_datasets["validation"]['target'], bart_preds_val)
bart_val_metrics

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'BLEU': 0.7015951162845684,
 'STA': 0.9178541492036881,
 'FLU': 0.71802455,
 'SEM': 0.9451393333184849,
 'Overall': 0.8400934599757737}

# Helper Functions to Fine-tune T5 Models

In [101]:
def add_prefix(datasetdict, prefix="to_neutral: "):
    """Adds a prefix to the source sequence in the dataset."""
    datasetdict_copy = datasetdict.copy()
    datasetdict_copy["train"] = datasetdict_copy["train"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["validation"] = datasetdict_copy["validation"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["test"] = datasetdict_copy["test"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy = DatasetDict(datasetdict_copy)
    return datasetdict_copy

def create_bidirectional_dataset(datasets, shuffle=True):
    """
    Creates a bi-directional dataset from the original dataset.

    Args:
        datasets (DatasetDict): DatasetDict object containing the original dataset.
        shuffle (bool): Whether to shuffle the dataset or not.
    
    Returns:
        extended_datasets (DatasetDict): DatasetDict object containing the bi-directional dataset.
    """

    def bidirectional_extension(dataset):
        new_data = {
            "source": [],
            "target": []
        }
        for src, tgt in zip(dataset['source'], dataset['target']):
            new_data['source'].extend([f'to_neutral: {src}', f'to_toxic: {tgt}'])
            new_data['target'].extend([tgt, src])
        return new_data

    extended_train_data = bidirectional_extension(datasets["train"])
    extended_validation_data = bidirectional_extension(datasets["validation"])
    extended_test_data = bidirectional_extension(datasets["test"])

    extended_datasets = DatasetDict({
        "train": Dataset.from_dict(extended_train_data),
        "validation": Dataset.from_dict(extended_validation_data),
        "test": Dataset.from_dict(extended_test_data)
    })

    if shuffle:
        extended_datasets["train"] = extended_datasets["train"].shuffle(seed=RANDOM_SEED)
        
    return extended_datasets

def preprocess_function(examples, tokenizer):
    """Preprocess function for T5."""
    model_inputs = tokenizer(
        examples["source"],
        text_target=examples["target"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
    )
    return model_inputs

def preprocess_dataset(dataset, tokenizer):
    """
    
    """

    return dataset.map(
        preprocess_function,
        fn_kwargs={'tokenizer': tokenizer},
        batched=True,
        remove_columns=["source", "target"],
    )

def post_process(preds, refs, tokenizer):
    """
    Post-process function for T5.

    Args:
        preds (list): list of predicted sequences
        refs (list): list of reference sequences
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding

    Returns:
        decoded_preds (list): list of decoded predicted sequences
        decoded_refs (list): list of decoded reference sequences
    """
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]

    return decoded_preds, decoded_refs

def compute_metrics(eval_preds, tokenizer):
    """
    Function to calculate the metrics for trainer.evaluate().

    Args:
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        eval_preds (tuple): Tuple containing the predictions and references

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # Evaluate metrics
    return evaluate_metrics(
        decoded_refs,
        decoded_preds,
        tokenizer_toxicity=tokenizer_toxicity,
        model_toxicity=model_toxicity,
        tokenizer_acceptability=tokenizer_acceptability,
        model_acceptability=model_acceptability,
        include_bleurt=False
    )

def compute_metrics_bd(eval_preds, tokenizer, bd_dataset, shuffled_data=False):
    """
    Function to calculate the metrics for trainer.evaluate().
    This function is for the bi-directional model.
    
    Args:
        eval_preds (tuple): Tuple containing the predictions and references
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        shuffled_data (bool): Whether the data is shuffled or not
        bd_dataset (DatasetDict): Bidirectional dataset to use for testing created using create_bidirectional_datasets
                                  For example, raw_datasets_bd["validation"] or raw_datasets_bd["test"]

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # If shuffled data is false, have to_neutral_preds and to_neutral_refs just be predictions and refs with even indices
    if not shuffled_data:
        to_neutral_preds = decoded_preds[::2]
        to_neutral_refs = decoded_refs[::2]
    # Otherwise, get the indices to use when splitting predictions and refs to to_neutral and to_toxic
    else:
        # Get the indices to use when splitting predictions and refs to to_neutral and to_toxic
        to_neutral_idx = [i for i, input_sentence in enumerate(bd_dataset['source']) if input_sentence.startswith("to_neutral")]

        # Retrieve based on the indices
        to_neutral_preds = [decoded_preds[i] for i in to_neutral_idx]
        to_neutral_refs = [decoded_refs[i] for i in to_neutral_idx]
    
    # Evaluate metrics for to_neutral
    to_neutral_metrics = evaluate_metrics(
        to_neutral_refs,
        to_neutral_preds,
    )

    # Return dictionary of to_neutral metrics
    return to_neutral_metrics

def setup_trainer(output_dir_name,
                train_dataset,
                eval_dataset,
                compute_metrics,
                model_checkpoint="t5-small",
                per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                num_train_epochs=NUM_TRAIN_EPOCHS,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=NUM_BEAMS,
                early_stopping_patience=EARLY_STOPPING_PATIENCE,
                report_to="wandb",
                ):
    """
    Set up a Seq2SeqTrainer object for training a T5 model.

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Evaluation dataset.
        compute_metrics (function): Function to compute metrics. Change this to compute_metrics_bd if using a bi-directional model.
        model_checkpoint (str): Model checkpoint to use.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of the output sequence.
        num_beams (int): Number of beams for beam search.
        early_stopping_patience (int): Number of epochs to wait before early stopping.
        report_to (str): Where to report results to. Either "wandb" or "none".

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    gen_config_path = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(gen_config_path)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=gen_config_path,
        fp16=True,
        report_to=report_to,
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )
   
    # Instantiate the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]

    )

    return trainer

def training_pipeline(model_name, project_name="t5-detox", model_checkpoint="t5-small", use_validation=True, raw_datasets=raw_datasets, bidirectional=False, shuffle=False, do_train=True):
    """
    Pipeline for training a T5 model. Saves the best model checkpoint to a txt file. Can also be used for evaluating a model (use test set instead of validation set).

    Args:
        model_name (str): Name of the model to name the output directory and wandb run.
        project_name (str): Name of the wandb project.
        model_checkpoint (str): Model checkpoint to use.
        use_validation (bool): Whether to use the validation set or not.
        raw_datasets (DatasetDict): DatasetDict object containing the original dataset.
        bidirectional (bool): Whether to use a bi-directional model or not.
        shuffle (bool): Whether to shuffle the dataset or not.
        do_train (bool): Whether to train the model or not.

    Returns:
        trainer (Seq2SeqTrainer): Trainer object for training the T5 model.
    """
    
    # Preprocess dataset (add prefixes / make bidirectional)
    if bidirectional:
        raw_datasets = create_bidirectional_dataset(raw_datasets, shuffle=shuffle)
    else:
        raw_datasets = add_prefix(raw_datasets)

    # Tokenize dataset
    tokenized_datasets = preprocess_dataset(raw_datasets, tokenizer_t5_small)

    # Define compute_metrics function depending on bidirectional or not
    if bidirectional and use_validation:
        bd_dataset = raw_datasets["validation"]
    elif bidirectional and not use_validation:
        bd_dataset = raw_datasets["test"]
    else:
        bd_dataset = None

    compute_metrics_fn = partial(compute_metrics_bd, bd_dataset=bd_dataset, shuffled_data=shuffle) if bd_dataset else compute_metrics

    # Setup trainer
    trainer = setup_trainer(
        output_dir_name=model_name,
        model_checkpoint=model_checkpoint,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"] if use_validation else tokenized_datasets["test"],
        compute_metrics=compute_metrics_fn
    )

    if do_train:
        # Initialize wandb
        wandb.init(project=project_name, name=model_name)
        trainer.train()
        wandb.finish()

        # Get the best checkpoint path for the model
        checkpoint_path = trainer.state.best_model_checkpoint

        # Save the checkpoint path for the best model
        with open(BEST_MODEL_CHECKPOINT_PATH, "a") as file:
            file.write(f"{model_name}: {checkpoint_path}\n")

    return trainer

# Fine-tune T5 (Unidirectional)

In [31]:
trainer_ud = training_pipeline(
    model_name="t5_small_unidir",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=raw_datasets,
    bidirectional=False,
    shuffle=False,
    do_train=True
)

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.181400,0.950930,0.589415,0.833194,0.691846,0.923100,0.774150
2,0.988400,0.929747,0.596775,0.856664,0.695719,0.924650,0.786094
3,0.912700,0.922288,0.596999,0.866723,0.692071,0.925726,0.789648
4,0.852600,0.918338,0.603083,0.885163,0.703110,0.925951,0.800494
5,0.806200,0.915330,0.606150,0.902766,0.707668,0.925923,0.809055
6,0.765900,0.931547,0.604702,0.891869,0.701773,0.926318,0.803306
7,0.732500,0.938001,0.599846,0.891031,0.706182,0.926642,0.802946


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▄▄▇█▇▅
eval/FLU,▁▃▁▆█▅▇
eval/Overall,▁▃▄▆█▇▇
eval/SEM,▁▄▆▇▇▇█
eval/STA,▁▃▄▆█▇▇
eval/loss,█▄▂▂▁▄▅
eval/runtime,█▇▄▇▁▂▆
eval/samples_per_second,▁▂▅▂█▇▃
eval/steps_per_second,▁▃▆▁██▃
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███


# Fine-tune T5 Model (Bi-directional, No custom loss)

### Trial without shuffled data

In [32]:
trainer_bd_ns = training_pipeline(
    model_name="t5_small_bidir_noshuf",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=raw_datasets,
    bidirectional=True,
    shuffle=False,
    do_train=True
)

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.487700,1.235429,0.589117,0.784577,0.678174,0.923197,0.751928
2,1.257100,1.167136,0.599725,0.836547,0.691290,0.925403,0.777902
3,1.171800,1.151006,0.600607,0.860855,0.689214,0.925960,0.787498
4,1.110400,1.143344,0.605002,0.892707,0.692520,0.926318,0.801851
5,1.063400,1.128787,0.604396,0.903604,0.704711,0.926202,0.808504
6,1.024400,1.140509,0.602471,0.898575,0.700976,0.925787,0.805277
7,0.989600,1.132926,0.601382,0.909472,0.705822,0.925794,0.810388
8,0.956400,1.134517,0.600770,0.900251,0.702148,0.925799,0.805844
9,0.930500,1.137347,0.604801,0.901090,0.699633,0.926039,0.806531


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▆▆██▇▆▆█
eval/FLU,▁▄▄▅█▇█▇▆
eval/Overall,▁▄▅▇█▇█▇█
eval/SEM,▁▆▇██▇▇▇▇
eval/STA,▁▄▅▇█▇█▇█
eval/loss,█▄▂▂▁▂▁▁▂
eval/runtime,▁▇█▆▇▄▅▆▅
eval/samples_per_second,█▂▁▃▂▅▄▃▃
eval/steps_per_second,█▂▁▃▂▅▃▃▃
train/epoch,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███


### Trial with shuffled data

In [33]:
trainer_bd_s = training_pipeline(
    model_name="t5_small_bidir_shuf",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=raw_datasets,
    bidirectional=True,
    shuffle=True,
    do_train=True
)

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.495300,1.230033,0.596087,0.780386,0.690938,0.922194,0.753998
2,1.258700,1.176923,0.599054,0.850796,0.688809,0.924131,0.782717
3,1.170300,1.152364,0.604108,0.879296,0.707586,0.923768,0.798811
4,1.110400,1.140243,0.607092,0.881811,0.704174,0.925567,0.800091
5,1.064300,1.131201,0.604929,0.887678,0.696949,0.925645,0.800576
6,1.021000,1.137493,0.605037,0.900251,0.705234,0.926110,0.807377
7,0.986400,1.132168,0.605474,0.914501,0.712354,0.925643,0.814495
8,0.954200,1.133756,0.599532,0.911148,0.709170,0.925135,0.811227
9,0.928600,1.135228,0.611682,0.912825,0.711991,0.926827,0.815230
10,0.903500,1.142378,0.607924,0.902766,0.704527,0.926615,0.808920


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▂▅▆▅▅▅▃█▆▅
eval/FLU,▂▁▇▆▃▆█▇█▆▇
eval/Overall,▁▄▆▆▆▇███▇█
eval/SEM,▁▄▃▆▆▇▆▅██▆
eval/STA,▁▅▆▆▇▇███▇█
eval/loss,█▄▂▂▁▁▁▁▁▂▂
eval/runtime,▃▃▃▃█▃▄▃▄▁▃
eval/samples_per_second,▆▅▅▅▁▆▅▆▄█▆
eval/steps_per_second,▆▅▅▅▁▆▄▆▄█▆
train/epoch,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇███


# Fine-tune T5 Model (Data Augmentation)

### All filters

In [34]:
trainer_aug_all = training_pipeline(
    model_name="t5_small_aug_all",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=aug_datasets_all_filters,
    bidirectional=False,
    shuffle=False,
    do_train=True
)

Map:   0%|          | 0/20711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.032800,0.949773,0.590953,0.870075,0.695258,0.924241,0.790121
2,0.879400,0.935219,0.590480,0.887678,0.702628,0.925248,0.798742
3,0.804800,0.929614,0.593576,0.892707,0.705193,0.925695,0.801976
4,0.746700,0.937668,0.589856,0.900251,0.705696,0.924643,0.804139
5,0.698400,0.941970,0.590971,0.893546,0.707449,0.926075,0.802317
6,0.658100,0.943837,0.589302,0.911987,0.722105,0.924957,0.812067
7,0.622200,0.961058,0.593418,0.896060,0.704455,0.925313,0.803061
8,0.592200,0.972355,0.593218,0.916178,0.714662,0.925547,0.813157
9,0.565100,0.985333,0.587340,0.906957,0.716202,0.924817,0.808455
10,0.539300,0.998677,0.589139,0.910310,0.716812,0.925013,0.810317


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▅▅█▄▅▃██▁▃
eval/FLU,▁▃▄▄▄█▃▆▆▇
eval/Overall,▁▄▅▅▅█▅█▇▇
eval/SEM,▁▅▇▃█▄▅▆▃▄
eval/STA,▁▄▄▆▅▇▅█▇▇
eval/loss,▃▂▁▂▂▂▄▅▇█
eval/runtime,▃█▆▇▅▅▂▅▇▁
eval/samples_per_second,▆▁▃▂▄▄▇▄▂█
eval/steps_per_second,▆▁▃▂▃▄▇▄▂█
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███


### No acceptability filter

In [35]:
trainer_aug_no_acc = training_pipeline(
    model_name="t5_small_aug_noaccept",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=aug_datasets_no_acceptability_filter,
    bidirectional=False,
    shuffle=False,
    do_train=True
)

Map:   0%|          | 0/20711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.076800,0.949558,0.589193,0.874267,0.699519,0.924891,0.792427
2,0.916100,0.936198,0.588487,0.884325,0.701398,0.925719,0.796851
3,0.839900,0.929998,0.595734,0.897737,0.708047,0.925757,0.805002
4,0.782200,0.937293,0.594991,0.901090,0.708011,0.924540,0.805944
5,0.734500,0.944445,0.595025,0.903604,0.709992,0.925311,0.807508
6,0.692700,0.948201,0.586651,0.899413,0.713622,0.925139,0.804848
7,0.657200,0.964795,0.587344,0.900251,0.714377,0.925392,0.805523


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▃▂█▇▇▁▂
eval/FLU,▁▂▅▅▆██
eval/Overall,▁▃▇▇█▇▇
eval/SEM,▃██▁▅▄▆
eval/STA,▁▃▇▇█▇▇
eval/loss,▅▂▁▂▄▅█
eval/runtime,▁▄▄▄█▅▄
eval/samples_per_second,█▄▅▅▁▄▅
eval/steps_per_second,█▅▅▅▁▄▅
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███


### No similarity Filter

In [36]:
trainer_aug_no_sim = training_pipeline(
    model_name="t5_small_aug_nosim",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=aug_datasets_no_similarity_filter,
    bidirectional=False,
    shuffle=False,
    do_train=True
)

Map:   0%|          | 0/20711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.311000,0.954922,0.588395,0.849120,0.704302,0.923190,0.782825
2,1.137600,0.933897,0.590768,0.877619,0.705733,0.924761,0.795300
3,1.050900,0.926761,0.596456,0.906119,0.707856,0.924799,0.808270
4,0.983600,0.928941,0.597504,0.904443,0.703530,0.924731,0.806930
5,0.927500,0.939296,0.592530,0.908634,0.707966,0.925006,0.808554
6,0.877100,0.949860,0.593862,0.917016,0.715056,0.925457,0.813681
7,0.834100,0.964241,0.592720,0.903604,0.706382,0.926139,0.806490
8,0.796000,0.974892,0.591599,0.917854,0.719671,0.924972,0.814390
9,0.763300,0.985350,0.589331,0.909472,0.713934,0.924521,0.809346
10,0.734800,0.995770,0.597483,0.911148,0.715207,0.925603,0.812118


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▃▇█▄▅▄▃▂█
eval/FLU,▁▂▃▁▃▆▂█▆▆
eval/Overall,▁▄▇▆▇█▆█▇▇
eval/SEM,▁▅▅▅▅▆█▅▄▇
eval/STA,▁▄▇▇▇█▇█▇▇
eval/loss,▄▂▁▁▂▃▅▆▇█
eval/runtime,▇█▅▁█▃▅▆▁▃
eval/samples_per_second,▂▁▄█▁▆▄▃█▆
eval/steps_per_second,▂▁▄█▁▆▄▃█▆
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███


### No toxicity filter

In [37]:
trainer_aug_no_tox = training_pipeline(
    model_name="t5_small_aug_notox",
    project_name="t5-detox",
    model_checkpoint="t5-small",
    use_validation=True,
    raw_datasets=aug_datasets_no_toxicity_filter,
    bidirectional=False,
    shuffle=False,
    do_train=True
)

Map:   0%|          | 0/20711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.012500,0.952226,0.590286,0.843252,0.696211,0.924479,0.779496
2,0.869700,0.941054,0.594841,0.863370,0.701323,0.925570,0.789695
3,0.797800,0.929157,0.595369,0.886002,0.695625,0.926623,0.797924
4,0.742600,0.938356,0.590351,0.876781,0.699310,0.925064,0.793658
5,0.695900,0.941544,0.600925,0.886840,0.704962,0.926235,0.801160
6,0.656300,0.936969,0.598625,0.892707,0.713698,0.925555,0.804659
7,0.621500,0.967940,0.595544,0.887678,0.700756,0.925482,0.799428
8,0.590200,0.970850,0.591349,0.892707,0.708442,0.925324,0.802106


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▄▄▁█▆▄▂
eval/FLU,▁▃▁▂▅█▃▆
eval/Overall,▁▄▆▅▇█▇▇
eval/SEM,▁▅█▃▇▅▄▄
eval/STA,▁▄▇▆▇█▇█
eval/loss,▅▃▁▃▃▂██
eval/runtime,▁▆▄▂▇▃▆█
eval/samples_per_second,█▃▅▇▂▆▃▁
eval/steps_per_second,█▄▅▇▂▆▃▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███


# Fine-tune T5 (Unidirectional, with Negative Lexically Constrained Decoding)

Get the IDs using the appropriate tokenizer for the bad words

In [135]:
def get_bad_words_list(dataset, tokenizer=tokenizer_toxicity, model=model_toxicity, num_layers=3, top_k=3):
    """
    Gets the top k bad words for each sentence in the dataset.

    Args:
        dataset (list): List of sentences.
        tokenizer (PreTrainedTokenizer): Tokenizer to use (toxicity classifier).
        model (PreTrainedModel): Model to use (toxicity classifier).
        num_layers (int): Number of layers to use.
        top_k (int): Number of bad words to return.

    Returns:
        bad_words_list (list): List of lists of bad words.
    """    
    bad_words_list = []

    for sentence in dataset:
        # Tokenize sentence
        inputs = tokenizer(sentence, return_tensors="pt").to(DEVICE)
        input_ids = inputs["input_ids"]

        # Get attention scores
        attention = model(input_ids, output_attentions=True)['attentions']

        # Get the last num_layers layer attention scores and average them
        attention = torch.stack(attention[-num_layers:]).mean(0)

        # Average across each head
        attention = attention.mean(1)

        # Sum each row to get the attention score for each token
        attention = attention.mean(1)

        # Exclude separator tokens and punctuation
        token_list = input_ids.squeeze().tolist()
        punctuation_ids = {tokenizer.convert_tokens_to_ids(token) for token in string.punctuation}
        exclude_ids = set([tokenizer.bos_token_id, tokenizer.eos_token_id]) | punctuation_ids

        valid_indices = [i for i, token_id in enumerate(token_list) if token_id not in exclude_ids]

        # Filter out the valid indices from the attention scores
        valid_attention = attention.squeeze()[valid_indices]

        # Get the indices of the top k tokens with the highest attention scores among valid tokens
        top_indices = valid_attention.topk(top_k).indices.tolist()
        top_token_indices = [valid_indices[i] for i in top_indices]

        # Decode the tokens
        bad_words = [tokenizer.decode(token_list[index]).strip() for index in top_token_indices]

        bad_words_list.append(bad_words)

    return bad_words_list

def get_bad_word_ids(dataset,
                     tokenizer_toxicity=tokenizer_toxicity,
                     model_toxicity=model_toxicity,
                     tokenizer_t5=tokenizer_t5_small,
                     num_layers=3,
                     top_k=3):
    
    # Get list of bad words as identified using attention from toxicity classifier
    bad_words_list = get_bad_words_list(dataset, tokenizer_toxicity, model_toxicity, num_layers, top_k)

    # Convert each list of bad words to a string
    bad_words_str_list = [" ".join(bad_words) for bad_words in bad_words_list]

    # Encode the bad words using the T5 tokenizer encode
    bad_word_ids = [tokenizer_t5.encode(bad_words, add_special_tokens=False) for bad_words in bad_words_str_list]

    return bad_word_ids

In [136]:
get_bad_word_ids(raw_datasets["validation"]['source'])

[[720, 524, 3, 15, 51, 3, 189, 32],
 [96, 3, 89, 4636, 1131, 26],
 [720, 524, 25, 3, 9],
 [3, 89, 4636, 53, 278, 34],
 [3, 89, 15318, 3, 31, 7, 33],
 [3, 834, 3, 89, 4636, 3, 18],
 [3, 7, 10536, 1466, 3, 15294],
 [3, 7, 10536, 16, 16],
 [3, 4636, 7, 3, 7, 10536, 3, 7],
 [16497, 110, 12032, 872],
 [3, 7, 10536, 224, 3, 9],
 [3, 7, 10536, 3, 9, 888],
 [3, 89, 4636, 363, 24],
 [34, 4488, 79],
 [3, 7, 10536, 24, 16],
 [3, 7, 10536, 3, 189, 32, 466],
 [38, 7, 470, 10657],
 [3, 7, 10536, 14732, 405],
 [3, 4636, 7, 225, 3, 89],
 [3, 89, 4636, 38, 17112],
 [38, 7, 3, 7137, 470],
 [1001, 7, 73, 8715, 9357],
 [3, 7, 10536, 2219, 3, 9],
 [3, 7, 10536, 48, 103],
 [16497, 3, 7, 10536, 24],
 [3, 89, 4636, 77, 25, 28],
 [148, 43, 1780],
 [15708, 100, 1176],
 [3, 7, 10536, 3, 172, 14924],
 [3, 89, 4636, 53, 94, 7, 59],
 [38, 7, 39, 56],
 [15708, 1670, 3, 157],
 [3, 26, 3142, 25, 130],
 [39, 6660, 7683],
 [3, 7, 10536, 16497, 25],
 [3, 89, 4636, 53, 81, 3, 35],
 [3, 89, 4636, 53, 22784, 5301],
 [5591, 

Next we try experimenting with generation using the bad_words_ids

In [98]:
def get_model_checkpoints():
    # Get checkpoint values for the best models
    with open(BEST_MODEL_CHECKPOINT_PATH, "r") as f:
        best_model_checkpoints = f.readlines()

    # Convert to a dictionary
    best_model_checkpoints_dict = {}
    for line in best_model_checkpoints:
        model_name, checkpoint_path = line.split(": ")
        best_model_checkpoints_dict[model_name] = checkpoint_path.strip()

    return best_model_checkpoints_dict

model_checkpoints = get_model_checkpoints()

In [99]:
model_checkpoints

{'t5_small_unidir': '../models/t5_small_unidir/checkpoint-840',
 't5_small_bidir_noshuf': '../models/t5_small_bidir_noshuf/checkpoint-2352',
 't5_small_bidir_shuf': '../models/t5_small_bidir_shuf/checkpoint-3024',
 't5_small_aug_all': '../models/t5_small_aug_all/checkpoint-2592',
 't5_small_aug_noaccept': '../models/t5_small_aug_noaccept/checkpoint-1620',
 't5_small_aug_nosim': '../models/t5_small_aug_nosim/checkpoint-2592',
 't5_small_aug_notox': '../models/t5_small_aug_notox/checkpoint-1944'}

In [103]:
t5_ud = training_pipeline(
    model_name = "t5_small_unidir",
    project_name = "t5-detox",
    model_checkpoint = model_checkpoints["t5_small_unidir"],
    use_validation = True,
    raw_datasets = raw_datasets,
    bidirectional = False,
    shuffle = False,
    do_train = False   
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1193
})

In [124]:
inputs = t5_ud.tokenizer(
    raw_datasets["validation"]['source'],
    return_tensors="pt",
    padding=True
)

inputs

{'input_ids': tensor([[  391,    17,    82,  ...,     0,     0,     0],
        [   96,  1131,    26,  ...,     0,     0,     0],
        [ 6214,   102,   258,  ...,     0,     0,     0],
        ...,
        [  465,    25,    38,  ...,     0,     0,     0],
        [11456,     6,    68,  ...,     0,     0,     0],
        [20067,     3, 14312,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [126]:
# Move t5 ud model to cpu
t5_ud.model.to("cpu")

t5_ud.model.generate(
    **inputs,
    max_length=MAX_OUTPUT_LENGTH,
    num_beams=NUM_BEAMS,
    early_stopping=True,
)

tensor([[    0,    27,   333,  ...,     0,     0,     0],
        [    0,  1624,    26,  ...,     0,     0,     0],
        [    0,  6214,   102,  ...,     0,     0,     0],
        ...,
        [    0,    27,  3290,  ...,     0,     0,     0],
        [    0, 11456,     6,  ...,     0,     0,     0],
        [    0, 20067,     3,  ...,     0,     0,     0]])